1. сделать feature engineering
2. обучить любой классификатор (какой вам нравится)
3. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
4. применить random negative sampling для построения классификатора в новых условиях
5. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
6. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

df = pd.read_csv('./support/lection6_materials/bank-full.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [2]:
df.describe(include='object')

,job,marital,education,default,housing,loan,contact,month,poutcome,y
count,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211
unique,12,3,4,2,2,2,3,12,4,2
top,blue-collar,married,secondary,no,yes,no,cellular,may,unknown,no
freq,9732,27214,23202,44396,25130,37967,29285,13766,36959,39922


In [3]:
df['job'].unique()

array(['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student'], dtype=object)

In [4]:
df.groupby('job')['balance'].mean() 

job
admin.           1135.838909
blue-collar      1078.826654
entrepreneur     1521.470074
housemaid        1392.395161
management       1763.616832
retired          1984.215106
self-employed    1647.970868
services          997.088108
student          1388.060768
technician       1252.632092
unemployed       1521.745971
unknown          1772.357639
Name: balance, dtype: float64

In [37]:
df['balance'].describe() # можно заметить, что среднее по каждой профессии не превышает среднеквадратичное отклонение выборки
# поэтому можно сделать вывод, что категориальная переменая "профессия" не значима

count     45211.000000
mean       1362.272058
std        3044.765829
min       -8019.000000
25%          72.000000
50%         448.000000
75%        1428.000000
max      102127.000000
Name: balance, dtype: float64

In [6]:
df.drop('job', axis=1, inplace=True)

In [7]:
df['marital'].unique()

array(['married', 'single', 'divorced'], dtype=object)

In [8]:
df['marital'].replace({'married': 1, 'single': 0, 'divorced': 2}, inplace=True)
df

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,1,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,0,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,1,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,1,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,0,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,1,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,2,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,1,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,1,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [9]:
df['education'].unique()

array(['tertiary', 'secondary', 'unknown', 'primary'], dtype=object)

In [10]:
df['education'].replace({'unknown': 0, 'primary': 1, 'secondary': 2, 'tertiary': 3}, inplace=True)
df

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,1,3,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,0,2,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,1,2,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,1,0,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,0,0,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,1,3,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,2,1,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,1,2,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,1,2,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [11]:
df.replace({'no': 0, 'yes': 1}, inplace=True)
df

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,1,3,0,2143,1,0,unknown,5,may,261,1,-1,0,unknown,0
1,44,0,2,0,29,1,0,unknown,5,may,151,1,-1,0,unknown,0
2,33,1,2,0,2,1,1,unknown,5,may,76,1,-1,0,unknown,0
3,47,1,0,0,1506,1,0,unknown,5,may,92,1,-1,0,unknown,0
4,33,0,0,0,1,0,0,unknown,5,may,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,1,3,0,825,0,0,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,2,1,0,1729,0,0,cellular,17,nov,456,2,-1,0,unknown,1
45208,72,1,2,0,5715,0,0,cellular,17,nov,1127,5,184,3,success,1
45209,57,1,2,0,668,0,0,telephone,17,nov,508,4,-1,0,unknown,0


In [12]:
df.describe(include='object')

,contact,month,poutcome
count,45211,45211,45211
unique,3,12,4
top,cellular,may,unknown
freq,29285,13766,36959


In [13]:
df['contact'].unique()

array(['unknown', 'cellular', 'telephone'], dtype=object)

In [14]:
df.groupby('y').count()

,age,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
y,,,,,,,,,,,,,,,
0,39922,39922,39922,39922,39922,39922,39922,39922,39922,39922,39922,39922,39922,39922,39922
1,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289,5289


In [15]:
5289/39922

0.1324833425179099

In [16]:
df.groupby('contact')['y'].mean() # contact не значащий аргумент

contact
cellular     0.149189
telephone    0.134205
unknown      0.040707
Name: y, dtype: float64

In [17]:
df.drop('contact', axis=1, inplace=True)
df['poutcome'].unique()

array(['unknown', 'failure', 'other', 'success'], dtype=object)

In [18]:
df['poutcome'].replace({'unknown': 0, 'failure': 1, 'other': 2, 'success':3}, inplace=True)

In [19]:
df['month'].unique()

array(['may', 'jun', 'jul', 'aug', 'oct', 'nov', 'dec', 'jan', 'feb',
       'mar', 'apr', 'sep'], dtype=object)

In [20]:
months = {'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'oct': 10, 'nov': 11, 'dec': 12, 'jan': 1, 'feb': 2,
       'mar' : 3, 'apr': 4, 'sep': 9}
df['month'].replace(months, inplace=True)
df.describe()

,age,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,poutcome,y
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,0.832275,2.060516,0.018027,1362.272058,0.555838,0.160226,15.806419,6.144655,258.163080,2.763841,40.197828,0.580323,0.290062,0.116985
std,10.618762,0.608230,0.778704,0.133049,3044.765829,0.496878,0.366820,8.322476,2.408034,257.527812,3.098021,100.128746,2.303441,0.698469,0.321406
min,18.000000,0.000000,0.000000,0.000000,-8019.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
25%,33.000000,0.000000,2.000000,0.000000,72.000000,0.000000,0.000000,8.000000,5.000000,103.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
50%,39.000000,1.000000,2.000000,0.000000,448.000000,1.000000,0.000000,16.000000,6.000000,180.000000,2.000000,-1.000000,0.000000,0.000000,0.000000
75%,48.000000,1.000000,3.000000,0.000000,1428.000000,1.000000,0.000000,21.000000,8.000000,319.000000,3.000000,-1.000000,0.000000,0.000000,0.000000
max,95.000000,2.000000,3.000000,1.000000,102127.000000,1.000000,1.000000,31.000000,12.000000,4918.000000,63.000000,871.000000,275.000000,3.000000,1.000000


In [21]:
from sklearn.model_selection import train_test_split

values = df.drop('y', axis=1)
target = df['y']

X_train, X_test, y_train, y_test = train_test_split(values, target, test_size=0.2, random_state=42)

model = CatBoostClassifier()
model.fit(X_train, y_train)

Learning rate set to 0.04768
0:	learn: 0.6349164	total: 230ms	remaining: 3m 50s
1:	learn: 0.5843754	total: 284ms	remaining: 2m 21s
2:	learn: 0.5398931	total: 340ms	remaining: 1m 52s
3:	learn: 0.5072942	total: 388ms	remaining: 1m 36s
4:	learn: 0.4778858	total: 434ms	remaining: 1m 26s
5:	learn: 0.4477770	total: 480ms	remaining: 1m 19s
6:	learn: 0.4231958	total: 525ms	remaining: 1m 14s
7:	learn: 0.4026092	total: 567ms	remaining: 1m 10s
8:	learn: 0.3846819	total: 608ms	remaining: 1m 6s
9:	learn: 0.3681967	total: 650ms	remaining: 1m 4s
10:	learn: 0.3560084	total: 696ms	remaining: 1m 2s
11:	learn: 0.3423433	total: 745ms	remaining: 1m 1s
12:	learn: 0.3292505	total: 791ms	remaining: 1m
13:	learn: 0.3196883	total: 831ms	remaining: 58.5s
14:	learn: 0.3098642	total: 877ms	remaining: 57.6s
15:	learn: 0.3028953	total: 922ms	remaining: 56.7s
16:	learn: 0.2953586	total: 964ms	remaining: 55.8s
17:	learn: 0.2886670	total: 1.01s	remaining: 55s
18:	learn: 0.2834394	total: 1.05s	remaining: 54.4s
19:	learn

165:	learn: 0.1988678	total: 5.35s	remaining: 26.9s
166:	learn: 0.1987753	total: 5.38s	remaining: 26.8s
167:	learn: 0.1986204	total: 5.41s	remaining: 26.8s
168:	learn: 0.1985146	total: 5.43s	remaining: 26.7s
169:	learn: 0.1984550	total: 5.46s	remaining: 26.7s
170:	learn: 0.1983485	total: 5.49s	remaining: 26.6s
171:	learn: 0.1982297	total: 5.52s	remaining: 26.6s
172:	learn: 0.1981022	total: 5.55s	remaining: 26.5s
173:	learn: 0.1979673	total: 5.58s	remaining: 26.5s
174:	learn: 0.1978841	total: 5.61s	remaining: 26.4s
175:	learn: 0.1977725	total: 5.63s	remaining: 26.4s
176:	learn: 0.1976492	total: 5.66s	remaining: 26.3s
177:	learn: 0.1975684	total: 5.69s	remaining: 26.3s
178:	learn: 0.1974664	total: 5.71s	remaining: 26.2s
179:	learn: 0.1974125	total: 5.75s	remaining: 26.2s
180:	learn: 0.1973291	total: 5.77s	remaining: 26.1s
181:	learn: 0.1972355	total: 5.8s	remaining: 26.1s
182:	learn: 0.1971304	total: 5.83s	remaining: 26s
183:	learn: 0.1969972	total: 5.86s	remaining: 26s
184:	learn: 0.196

328:	learn: 0.1834565	total: 10.1s	remaining: 20.6s
329:	learn: 0.1833965	total: 10.1s	remaining: 20.6s
330:	learn: 0.1833219	total: 10.2s	remaining: 20.6s
331:	learn: 0.1832507	total: 10.2s	remaining: 20.5s
332:	learn: 0.1831774	total: 10.2s	remaining: 20.5s
333:	learn: 0.1830837	total: 10.3s	remaining: 20.5s
334:	learn: 0.1830266	total: 10.3s	remaining: 20.4s
335:	learn: 0.1829802	total: 10.3s	remaining: 20.4s
336:	learn: 0.1829071	total: 10.4s	remaining: 20.4s
337:	learn: 0.1828043	total: 10.4s	remaining: 20.4s
338:	learn: 0.1827442	total: 10.4s	remaining: 20.3s
339:	learn: 0.1826827	total: 10.5s	remaining: 20.3s
340:	learn: 0.1826279	total: 10.5s	remaining: 20.3s
341:	learn: 0.1825468	total: 10.5s	remaining: 20.2s
342:	learn: 0.1824966	total: 10.5s	remaining: 20.2s
343:	learn: 0.1824832	total: 10.6s	remaining: 20.1s
344:	learn: 0.1824058	total: 10.6s	remaining: 20.1s
345:	learn: 0.1823188	total: 10.6s	remaining: 20.1s
346:	learn: 0.1821996	total: 10.6s	remaining: 20s
347:	learn: 0.

488:	learn: 0.1726613	total: 14.7s	remaining: 15.3s
489:	learn: 0.1726126	total: 14.7s	remaining: 15.3s
490:	learn: 0.1725438	total: 14.7s	remaining: 15.3s
491:	learn: 0.1724871	total: 14.7s	remaining: 15.2s
492:	learn: 0.1724190	total: 14.8s	remaining: 15.2s
493:	learn: 0.1723644	total: 14.8s	remaining: 15.2s
494:	learn: 0.1722928	total: 14.8s	remaining: 15.1s
495:	learn: 0.1722416	total: 14.9s	remaining: 15.1s
496:	learn: 0.1721821	total: 14.9s	remaining: 15.1s
497:	learn: 0.1721436	total: 14.9s	remaining: 15s
498:	learn: 0.1720831	total: 15s	remaining: 15s
499:	learn: 0.1720217	total: 15s	remaining: 15s
500:	learn: 0.1719666	total: 15s	remaining: 15s
501:	learn: 0.1718720	total: 15s	remaining: 14.9s
502:	learn: 0.1717953	total: 15.1s	remaining: 14.9s
503:	learn: 0.1717365	total: 15.1s	remaining: 14.9s
504:	learn: 0.1716713	total: 15.1s	remaining: 14.8s
505:	learn: 0.1716111	total: 15.2s	remaining: 14.8s
506:	learn: 0.1715795	total: 15.2s	remaining: 14.8s
507:	learn: 0.1715035	total:

649:	learn: 0.1648509	total: 19.2s	remaining: 10.3s
650:	learn: 0.1648160	total: 19.2s	remaining: 10.3s
651:	learn: 0.1647623	total: 19.2s	remaining: 10.3s
652:	learn: 0.1647261	total: 19.3s	remaining: 10.2s
653:	learn: 0.1646952	total: 19.3s	remaining: 10.2s
654:	learn: 0.1646462	total: 19.3s	remaining: 10.2s
655:	learn: 0.1645740	total: 19.4s	remaining: 10.1s
656:	learn: 0.1645072	total: 19.4s	remaining: 10.1s
657:	learn: 0.1644706	total: 19.4s	remaining: 10.1s
658:	learn: 0.1644173	total: 19.4s	remaining: 10.1s
659:	learn: 0.1643460	total: 19.5s	remaining: 10s
660:	learn: 0.1642851	total: 19.5s	remaining: 10s
661:	learn: 0.1642370	total: 19.5s	remaining: 9.97s
662:	learn: 0.1642045	total: 19.6s	remaining: 9.94s
663:	learn: 0.1641566	total: 19.6s	remaining: 9.91s
664:	learn: 0.1641074	total: 19.6s	remaining: 9.89s
665:	learn: 0.1640762	total: 19.7s	remaining: 9.86s
666:	learn: 0.1640067	total: 19.7s	remaining: 9.83s
667:	learn: 0.1639589	total: 19.7s	remaining: 9.8s
668:	learn: 0.163

808:	learn: 0.1576202	total: 23.8s	remaining: 5.62s
809:	learn: 0.1575780	total: 23.8s	remaining: 5.59s
810:	learn: 0.1575089	total: 23.9s	remaining: 5.56s
811:	learn: 0.1574547	total: 23.9s	remaining: 5.53s
812:	learn: 0.1574252	total: 23.9s	remaining: 5.5s
813:	learn: 0.1573374	total: 24s	remaining: 5.47s
814:	learn: 0.1572766	total: 24s	remaining: 5.44s
815:	learn: 0.1572318	total: 24s	remaining: 5.41s
816:	learn: 0.1571993	total: 24s	remaining: 5.38s
817:	learn: 0.1571472	total: 24.1s	remaining: 5.36s
818:	learn: 0.1571073	total: 24.1s	remaining: 5.33s
819:	learn: 0.1570720	total: 24.1s	remaining: 5.3s
820:	learn: 0.1570363	total: 24.2s	remaining: 5.27s
821:	learn: 0.1570037	total: 24.2s	remaining: 5.24s
822:	learn: 0.1569411	total: 24.2s	remaining: 5.21s
823:	learn: 0.1569115	total: 24.2s	remaining: 5.18s
824:	learn: 0.1568772	total: 24.3s	remaining: 5.15s
825:	learn: 0.1568230	total: 24.3s	remaining: 5.12s
826:	learn: 0.1567788	total: 24.3s	remaining: 5.09s
827:	learn: 0.1567381	

969:	learn: 0.1506904	total: 28.4s	remaining: 879ms
970:	learn: 0.1506468	total: 28.5s	remaining: 850ms
971:	learn: 0.1506129	total: 28.5s	remaining: 821ms
972:	learn: 0.1505750	total: 28.5s	remaining: 791ms
973:	learn: 0.1505284	total: 28.5s	remaining: 762ms
974:	learn: 0.1504791	total: 28.6s	remaining: 733ms
975:	learn: 0.1504333	total: 28.6s	remaining: 703ms
976:	learn: 0.1504102	total: 28.6s	remaining: 674ms
977:	learn: 0.1503710	total: 28.7s	remaining: 645ms
978:	learn: 0.1503018	total: 28.7s	remaining: 615ms
979:	learn: 0.1502814	total: 28.7s	remaining: 586ms
980:	learn: 0.1502403	total: 28.7s	remaining: 557ms
981:	learn: 0.1501911	total: 28.8s	remaining: 527ms
982:	learn: 0.1501466	total: 28.8s	remaining: 498ms
983:	learn: 0.1501214	total: 28.8s	remaining: 469ms
984:	learn: 0.1500808	total: 28.9s	remaining: 439ms
985:	learn: 0.1500568	total: 28.9s	remaining: 410ms
986:	learn: 0.1500135	total: 28.9s	remaining: 381ms
987:	learn: 0.1499857	total: 28.9s	remaining: 351ms
988:	learn: 

In [22]:
model.best_score_


results = {}

In [23]:
y_pred = model.predict(X_test)

from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    
    return [f1, roc, rec, prc]

    
results['simpl_model'] = evaluate_results(y_test, y_pred)

Classification results:
f1: 56.16%
roc: 72.70%
recall: 48.85%
precision: 66.05%


In [24]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 1323/5289 as positives and unlabeling the rest


In [25]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    43888
 1     1323
Name: class_test, dtype: int64


In [26]:
mod_data.head(10)

,age,marital,education,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,poutcome,y,class_test
0,58,1,3,0,2143,1,0,5,5,261,1,-1,0,0,0,-1
1,44,0,2,0,29,1,0,5,5,151,1,-1,0,0,0,-1
2,33,1,2,0,2,1,1,5,5,76,1,-1,0,0,0,-1
3,47,1,0,0,1506,1,0,5,5,92,1,-1,0,0,0,-1
4,33,0,0,0,1,0,0,5,5,198,1,-1,0,0,0,-1
5,35,1,3,0,231,1,0,5,5,139,1,-1,0,0,0,-1
6,28,0,3,0,447,1,1,5,5,217,1,-1,0,0,0,-1
7,42,2,3,1,2,1,0,5,5,380,1,-1,0,0,0,-1
8,58,1,1,0,121,1,0,5,5,50,1,-1,0,0,0,-1
9,43,0,2,0,593,1,0,5,5,55,1,-1,0,0,0,-1


In [27]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [28]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(1323, 16) (1323, 16)


In [29]:
model = CatBoostClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)


Learning rate set to 0.015609
0:	learn: 0.6859560	total: 6.92ms	remaining: 6.91s
1:	learn: 0.6782727	total: 14.4ms	remaining: 7.2s
2:	learn: 0.6708693	total: 21.9ms	remaining: 7.26s
3:	learn: 0.6636686	total: 28.8ms	remaining: 7.17s
4:	learn: 0.6568493	total: 35.7ms	remaining: 7.1s
5:	learn: 0.6502358	total: 41.9ms	remaining: 6.95s
6:	learn: 0.6434021	total: 48.5ms	remaining: 6.88s
7:	learn: 0.6364651	total: 54.6ms	remaining: 6.77s
8:	learn: 0.6305669	total: 60.8ms	remaining: 6.7s
9:	learn: 0.6245933	total: 66.8ms	remaining: 6.62s
10:	learn: 0.6189101	total: 73ms	remaining: 6.56s
11:	learn: 0.6131605	total: 79ms	remaining: 6.5s
12:	learn: 0.6081037	total: 85ms	remaining: 6.46s
13:	learn: 0.6025774	total: 91.3ms	remaining: 6.43s
14:	learn: 0.5971643	total: 97.3ms	remaining: 6.39s
15:	learn: 0.5929140	total: 104ms	remaining: 6.38s
16:	learn: 0.5880548	total: 111ms	remaining: 6.42s
17:	learn: 0.5833092	total: 118ms	remaining: 6.41s
18:	learn: 0.5784625	total: 124ms	remaining: 6.39s
19:	le

166:	learn: 0.3788143	total: 1.11s	remaining: 5.54s
167:	learn: 0.3782455	total: 1.12s	remaining: 5.54s
168:	learn: 0.3776500	total: 1.12s	remaining: 5.53s
169:	learn: 0.3772522	total: 1.13s	remaining: 5.52s
170:	learn: 0.3767346	total: 1.14s	remaining: 5.51s
171:	learn: 0.3762709	total: 1.14s	remaining: 5.51s
172:	learn: 0.3754166	total: 1.15s	remaining: 5.5s
173:	learn: 0.3749433	total: 1.16s	remaining: 5.5s
174:	learn: 0.3744042	total: 1.17s	remaining: 5.49s
175:	learn: 0.3740944	total: 1.17s	remaining: 5.49s
176:	learn: 0.3736849	total: 1.18s	remaining: 5.48s
177:	learn: 0.3731741	total: 1.19s	remaining: 5.47s
178:	learn: 0.3728042	total: 1.19s	remaining: 5.47s
179:	learn: 0.3725652	total: 1.2s	remaining: 5.46s
180:	learn: 0.3720230	total: 1.2s	remaining: 5.45s
181:	learn: 0.3716016	total: 1.21s	remaining: 5.44s
182:	learn: 0.3712624	total: 1.22s	remaining: 5.43s
183:	learn: 0.3708444	total: 1.22s	remaining: 5.42s
184:	learn: 0.3704386	total: 1.23s	remaining: 5.41s
185:	learn: 0.37

336:	learn: 0.3215870	total: 2.44s	remaining: 4.81s
337:	learn: 0.3213519	total: 2.45s	remaining: 4.8s
338:	learn: 0.3210942	total: 2.46s	remaining: 4.79s
339:	learn: 0.3209643	total: 2.47s	remaining: 4.79s
340:	learn: 0.3207758	total: 2.48s	remaining: 4.78s
341:	learn: 0.3205795	total: 2.48s	remaining: 4.78s
342:	learn: 0.3203271	total: 2.49s	remaining: 4.77s
343:	learn: 0.3202241	total: 2.5s	remaining: 4.76s
344:	learn: 0.3202211	total: 2.5s	remaining: 4.75s
345:	learn: 0.3200294	total: 2.51s	remaining: 4.74s
346:	learn: 0.3197677	total: 2.52s	remaining: 4.74s
347:	learn: 0.3193613	total: 2.52s	remaining: 4.73s
348:	learn: 0.3192276	total: 2.53s	remaining: 4.72s
349:	learn: 0.3190326	total: 2.54s	remaining: 4.71s
350:	learn: 0.3189573	total: 2.54s	remaining: 4.71s
351:	learn: 0.3188393	total: 2.55s	remaining: 4.7s
352:	learn: 0.3185763	total: 2.56s	remaining: 4.69s
353:	learn: 0.3182858	total: 2.56s	remaining: 4.68s
354:	learn: 0.3180908	total: 2.57s	remaining: 4.67s
355:	learn: 0.31

505:	learn: 0.2879639	total: 3.97s	remaining: 3.88s
506:	learn: 0.2876948	total: 3.98s	remaining: 3.87s
507:	learn: 0.2875544	total: 3.99s	remaining: 3.87s
508:	learn: 0.2872642	total: 4s	remaining: 3.86s
509:	learn: 0.2872146	total: 4.02s	remaining: 3.86s
510:	learn: 0.2870614	total: 4.03s	remaining: 3.85s
511:	learn: 0.2868073	total: 4.04s	remaining: 3.85s
512:	learn: 0.2866018	total: 4.06s	remaining: 3.85s
513:	learn: 0.2865634	total: 4.07s	remaining: 3.85s
514:	learn: 0.2864425	total: 4.08s	remaining: 3.85s
515:	learn: 0.2863209	total: 4.1s	remaining: 3.84s
516:	learn: 0.2860968	total: 4.11s	remaining: 3.84s
517:	learn: 0.2858450	total: 4.12s	remaining: 3.83s
518:	learn: 0.2857355	total: 4.13s	remaining: 3.83s
519:	learn: 0.2856498	total: 4.14s	remaining: 3.82s
520:	learn: 0.2854524	total: 4.15s	remaining: 3.82s
521:	learn: 0.2853061	total: 4.16s	remaining: 3.81s
522:	learn: 0.2850115	total: 4.17s	remaining: 3.81s
523:	learn: 0.2848540	total: 4.18s	remaining: 3.8s
524:	learn: 0.284

668:	learn: 0.2585557	total: 5.83s	remaining: 2.88s
669:	learn: 0.2584659	total: 5.84s	remaining: 2.88s
670:	learn: 0.2582654	total: 5.86s	remaining: 2.87s
671:	learn: 0.2579973	total: 5.87s	remaining: 2.86s
672:	learn: 0.2577566	total: 5.88s	remaining: 2.86s
673:	learn: 0.2576718	total: 5.89s	remaining: 2.85s
674:	learn: 0.2575599	total: 5.9s	remaining: 2.84s
675:	learn: 0.2574738	total: 5.92s	remaining: 2.83s
676:	learn: 0.2573061	total: 5.93s	remaining: 2.83s
677:	learn: 0.2571093	total: 5.94s	remaining: 2.82s
678:	learn: 0.2570399	total: 5.95s	remaining: 2.81s
679:	learn: 0.2569158	total: 5.96s	remaining: 2.81s
680:	learn: 0.2567577	total: 5.97s	remaining: 2.8s
681:	learn: 0.2566610	total: 5.98s	remaining: 2.79s
682:	learn: 0.2564694	total: 5.99s	remaining: 2.78s
683:	learn: 0.2563429	total: 6s	remaining: 2.77s
684:	learn: 0.2561560	total: 6.02s	remaining: 2.77s
685:	learn: 0.2559886	total: 6.03s	remaining: 2.76s
686:	learn: 0.2557887	total: 6.04s	remaining: 2.75s
687:	learn: 0.255

831:	learn: 0.2335976	total: 7.67s	remaining: 1.55s
832:	learn: 0.2334758	total: 7.68s	remaining: 1.54s
833:	learn: 0.2333490	total: 7.69s	remaining: 1.53s
834:	learn: 0.2331563	total: 7.7s	remaining: 1.52s
835:	learn: 0.2329969	total: 7.71s	remaining: 1.51s
836:	learn: 0.2327584	total: 7.71s	remaining: 1.5s
837:	learn: 0.2326300	total: 7.72s	remaining: 1.49s
838:	learn: 0.2325132	total: 7.73s	remaining: 1.48s
839:	learn: 0.2323365	total: 7.73s	remaining: 1.47s
840:	learn: 0.2321792	total: 7.74s	remaining: 1.46s
841:	learn: 0.2320071	total: 7.74s	remaining: 1.45s
842:	learn: 0.2318391	total: 7.75s	remaining: 1.44s
843:	learn: 0.2316593	total: 7.75s	remaining: 1.43s
844:	learn: 0.2314691	total: 7.76s	remaining: 1.42s
845:	learn: 0.2313688	total: 7.77s	remaining: 1.41s
846:	learn: 0.2312324	total: 7.77s	remaining: 1.4s
847:	learn: 0.2311323	total: 7.78s	remaining: 1.39s
848:	learn: 0.2310459	total: 7.79s	remaining: 1.39s
849:	learn: 0.2308599	total: 7.79s	remaining: 1.38s
850:	learn: 0.2

990:	learn: 0.2130654	total: 9.03s	remaining: 82ms
991:	learn: 0.2129277	total: 9.04s	remaining: 72.9ms
992:	learn: 0.2127845	total: 9.04s	remaining: 63.8ms
993:	learn: 0.2126845	total: 9.05s	remaining: 54.7ms
994:	learn: 0.2125893	total: 9.06s	remaining: 45.6ms
995:	learn: 0.2124940	total: 9.07s	remaining: 36.4ms
996:	learn: 0.2124033	total: 9.08s	remaining: 27.3ms
997:	learn: 0.2123001	total: 9.09s	remaining: 18.2ms
998:	learn: 0.2122151	total: 9.1s	remaining: 9.11ms
999:	learn: 0.2121075	total: 9.11s	remaining: 0us


In [30]:
results['RNS_25'] = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 48.06%
roc: 85.26%
recall: 88.41%
precision: 32.99%


In [31]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.10 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

model = CatBoostClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

Using 529/5289 as positives and unlabeling the rest
target variable:
 -1    44682
 1      529
Name: class_test, dtype: int64
(529, 16) (529, 16)
Learning rate set to 0.010553
0:	learn: 0.6867868	total: 5.91ms	remaining: 5.91s
1:	learn: 0.6799554	total: 11.8ms	remaining: 5.87s
2:	learn: 0.6734751	total: 17.8ms	remaining: 5.92s
3:	learn: 0.6665673	total: 22.3ms	remaining: 5.56s
4:	learn: 0.6619724	total: 27ms	remaining: 5.38s
5:	learn: 0.6563225	total: 32.6ms	remaining: 5.39s
6:	learn: 0.6500441	total: 37.4ms	remaining: 5.3s
7:	learn: 0.6453465	total: 42.2ms	remaining: 5.24s
8:	learn: 0.6413559	total: 48.5ms	remaining: 5.34s
9:	learn: 0.6356264	total: 54.5ms	remaining: 5.39s
10:	learn: 0.6317756	total: 59.6ms	remaining: 5.36s
11:	learn: 0.6272879	total: 64.8ms	remaining: 5.33s
12:	learn: 0.6235279	total: 70.7ms	remaining: 5.37s
13:	learn: 0.6183298	total: 76.2ms	remaining: 5.37s
14:	learn: 0.6130044	total: 81.4ms	remaining: 5.35s
15:	learn: 0.6084819	total: 86.6ms	remaining: 5.32s
16:	le

173:	learn: 0.3520232	total: 1.02s	remaining: 4.84s
174:	learn: 0.3511679	total: 1.03s	remaining: 4.84s
175:	learn: 0.3505540	total: 1.03s	remaining: 4.83s
176:	learn: 0.3499804	total: 1.04s	remaining: 4.83s
177:	learn: 0.3491198	total: 1.04s	remaining: 4.82s
178:	learn: 0.3488387	total: 1.05s	remaining: 4.82s
179:	learn: 0.3482924	total: 1.05s	remaining: 4.81s
180:	learn: 0.3475668	total: 1.06s	remaining: 4.8s
181:	learn: 0.3470512	total: 1.07s	remaining: 4.8s
182:	learn: 0.3464418	total: 1.07s	remaining: 4.79s
183:	learn: 0.3459952	total: 1.08s	remaining: 4.79s
184:	learn: 0.3453373	total: 1.09s	remaining: 4.79s
185:	learn: 0.3448183	total: 1.09s	remaining: 4.79s
186:	learn: 0.3443718	total: 1.1s	remaining: 4.78s
187:	learn: 0.3437680	total: 1.1s	remaining: 4.77s
188:	learn: 0.3433746	total: 1.11s	remaining: 4.77s
189:	learn: 0.3429900	total: 1.12s	remaining: 4.76s
190:	learn: 0.3424902	total: 1.12s	remaining: 4.76s
191:	learn: 0.3420625	total: 1.13s	remaining: 4.76s
192:	learn: 0.34

353:	learn: 0.2802503	total: 2.31s	remaining: 4.21s
354:	learn: 0.2799355	total: 2.32s	remaining: 4.21s
355:	learn: 0.2797563	total: 2.33s	remaining: 4.21s
356:	learn: 0.2795671	total: 2.33s	remaining: 4.2s
357:	learn: 0.2792331	total: 2.34s	remaining: 4.2s
358:	learn: 0.2790036	total: 2.35s	remaining: 4.19s
359:	learn: 0.2788660	total: 2.35s	remaining: 4.19s
360:	learn: 0.2784369	total: 2.36s	remaining: 4.18s
361:	learn: 0.2781928	total: 2.37s	remaining: 4.18s
362:	learn: 0.2778319	total: 2.38s	remaining: 4.17s
363:	learn: 0.2776745	total: 2.39s	remaining: 4.17s
364:	learn: 0.2772288	total: 2.39s	remaining: 4.16s
365:	learn: 0.2769768	total: 2.4s	remaining: 4.16s
366:	learn: 0.2768215	total: 2.41s	remaining: 4.15s
367:	learn: 0.2765325	total: 2.42s	remaining: 4.15s
368:	learn: 0.2762002	total: 2.42s	remaining: 4.14s
369:	learn: 0.2759110	total: 2.43s	remaining: 4.14s
370:	learn: 0.2756721	total: 2.44s	remaining: 4.13s
371:	learn: 0.2754582	total: 2.44s	remaining: 4.13s
372:	learn: 0.2

538:	learn: 0.2366305	total: 3.6s	remaining: 3.08s
539:	learn: 0.2365773	total: 3.6s	remaining: 3.07s
540:	learn: 0.2363034	total: 3.61s	remaining: 3.06s
541:	learn: 0.2361639	total: 3.61s	remaining: 3.05s
542:	learn: 0.2359342	total: 3.62s	remaining: 3.05s
543:	learn: 0.2358035	total: 3.62s	remaining: 3.04s
544:	learn: 0.2355606	total: 3.63s	remaining: 3.03s
545:	learn: 0.2352789	total: 3.63s	remaining: 3.02s
546:	learn: 0.2351447	total: 3.64s	remaining: 3.01s
547:	learn: 0.2350362	total: 3.64s	remaining: 3s
548:	learn: 0.2348483	total: 3.65s	remaining: 3s
549:	learn: 0.2346890	total: 3.65s	remaining: 2.99s
550:	learn: 0.2345660	total: 3.66s	remaining: 2.98s
551:	learn: 0.2343490	total: 3.66s	remaining: 2.97s
552:	learn: 0.2340876	total: 3.67s	remaining: 2.96s
553:	learn: 0.2338662	total: 3.67s	remaining: 2.96s
554:	learn: 0.2335489	total: 3.68s	remaining: 2.95s
555:	learn: 0.2333509	total: 3.68s	remaining: 2.94s
556:	learn: 0.2331487	total: 3.69s	remaining: 2.93s
557:	learn: 0.233013

698:	learn: 0.2061080	total: 4.73s	remaining: 2.04s
699:	learn: 0.2060695	total: 4.74s	remaining: 2.03s
700:	learn: 0.2059870	total: 4.75s	remaining: 2.02s
701:	learn: 0.2057588	total: 4.76s	remaining: 2.02s
702:	learn: 0.2056429	total: 4.76s	remaining: 2.01s
703:	learn: 0.2055169	total: 4.77s	remaining: 2.01s
704:	learn: 0.2053699	total: 4.78s	remaining: 2s
705:	learn: 0.2052105	total: 4.79s	remaining: 1.99s
706:	learn: 0.2051140	total: 4.8s	remaining: 1.99s
707:	learn: 0.2049666	total: 4.8s	remaining: 1.98s
708:	learn: 0.2048647	total: 4.81s	remaining: 1.98s
709:	learn: 0.2047499	total: 4.82s	remaining: 1.97s
710:	learn: 0.2045954	total: 4.83s	remaining: 1.96s
711:	learn: 0.2044545	total: 4.83s	remaining: 1.96s
712:	learn: 0.2041849	total: 4.84s	remaining: 1.95s
713:	learn: 0.2040485	total: 4.85s	remaining: 1.94s
714:	learn: 0.2037521	total: 4.86s	remaining: 1.94s
715:	learn: 0.2034912	total: 4.86s	remaining: 1.93s
716:	learn: 0.2033832	total: 4.87s	remaining: 1.92s
717:	learn: 0.203

865:	learn: 0.1785477	total: 6.04s	remaining: 934ms
866:	learn: 0.1784210	total: 6.04s	remaining: 927ms
867:	learn: 0.1783669	total: 6.05s	remaining: 921ms
868:	learn: 0.1782366	total: 6.06s	remaining: 914ms
869:	learn: 0.1781426	total: 6.07s	remaining: 907ms
870:	learn: 0.1779715	total: 6.08s	remaining: 900ms
871:	learn: 0.1778415	total: 6.08s	remaining: 893ms
872:	learn: 0.1777538	total: 6.09s	remaining: 886ms
873:	learn: 0.1776436	total: 6.1s	remaining: 879ms
874:	learn: 0.1774141	total: 6.11s	remaining: 872ms
875:	learn: 0.1773038	total: 6.12s	remaining: 866ms
876:	learn: 0.1771471	total: 6.13s	remaining: 860ms
877:	learn: 0.1769966	total: 6.14s	remaining: 853ms
878:	learn: 0.1768404	total: 6.14s	remaining: 846ms
879:	learn: 0.1766615	total: 6.15s	remaining: 839ms
880:	learn: 0.1765062	total: 6.16s	remaining: 832ms
881:	learn: 0.1764209	total: 6.17s	remaining: 825ms
882:	learn: 0.1761646	total: 6.18s	remaining: 818ms
883:	learn: 0.1759877	total: 6.18s	remaining: 811ms
884:	learn: 0

In [33]:
results['RNS_10'] = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 50.25%
roc: 83.90%
recall: 86.59%
precision: 35.40%


In [34]:
mod_data = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.40 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

model = CatBoostClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

Using 2116/5289 as positives and unlabeling the rest
target variable:
 -1    43095
 1     2116
Name: class_test, dtype: int64
(2116, 16) (2116, 16)
Learning rate set to 0.019075
0:	learn: 0.6818524	total: 8.64ms	remaining: 8.63s
1:	learn: 0.6714053	total: 17.2ms	remaining: 8.57s
2:	learn: 0.6622736	total: 25.2ms	remaining: 8.38s
3:	learn: 0.6518069	total: 33.7ms	remaining: 8.4s
4:	learn: 0.6417274	total: 42.6ms	remaining: 8.47s
5:	learn: 0.6333241	total: 51.6ms	remaining: 8.55s
6:	learn: 0.6251979	total: 60.3ms	remaining: 8.56s
7:	learn: 0.6174495	total: 69.2ms	remaining: 8.58s
8:	learn: 0.6097299	total: 77.5ms	remaining: 8.53s
9:	learn: 0.6026367	total: 86.1ms	remaining: 8.53s
10:	learn: 0.5949658	total: 94.4ms	remaining: 8.48s
11:	learn: 0.5877993	total: 103ms	remaining: 8.48s
12:	learn: 0.5816322	total: 112ms	remaining: 8.5s
13:	learn: 0.5752317	total: 122ms	remaining: 8.56s
14:	learn: 0.5682175	total: 130ms	remaining: 8.55s
15:	learn: 0.5634241	total: 140ms	remaining: 8.63s
16:	lea

159:	learn: 0.3587291	total: 1.42s	remaining: 7.46s
160:	learn: 0.3583166	total: 1.43s	remaining: 7.44s
161:	learn: 0.3578518	total: 1.44s	remaining: 7.43s
162:	learn: 0.3571003	total: 1.45s	remaining: 7.42s
163:	learn: 0.3565584	total: 1.45s	remaining: 7.42s
164:	learn: 0.3562123	total: 1.46s	remaining: 7.41s
165:	learn: 0.3557992	total: 1.47s	remaining: 7.41s
166:	learn: 0.3551489	total: 1.48s	remaining: 7.4s
167:	learn: 0.3547601	total: 1.49s	remaining: 7.39s
168:	learn: 0.3541374	total: 1.5s	remaining: 7.38s
169:	learn: 0.3537656	total: 1.51s	remaining: 7.38s
170:	learn: 0.3534092	total: 1.52s	remaining: 7.38s
171:	learn: 0.3527649	total: 1.53s	remaining: 7.37s
172:	learn: 0.3524572	total: 1.54s	remaining: 7.36s
173:	learn: 0.3519820	total: 1.55s	remaining: 7.35s
174:	learn: 0.3516411	total: 1.56s	remaining: 7.36s
175:	learn: 0.3511360	total: 1.57s	remaining: 7.35s
176:	learn: 0.3508037	total: 1.58s	remaining: 7.35s
177:	learn: 0.3502944	total: 1.59s	remaining: 7.34s
178:	learn: 0.

331:	learn: 0.3119141	total: 2.95s	remaining: 5.95s
332:	learn: 0.3116616	total: 2.96s	remaining: 5.94s
333:	learn: 0.3113072	total: 2.98s	remaining: 5.93s
334:	learn: 0.3110964	total: 2.99s	remaining: 5.93s
335:	learn: 0.3109822	total: 3s	remaining: 5.92s
336:	learn: 0.3107439	total: 3.01s	remaining: 5.92s
337:	learn: 0.3106303	total: 3.02s	remaining: 5.91s
338:	learn: 0.3103181	total: 3.03s	remaining: 5.91s
339:	learn: 0.3101286	total: 3.04s	remaining: 5.9s
340:	learn: 0.3099143	total: 3.05s	remaining: 5.9s
341:	learn: 0.3098407	total: 3.07s	remaining: 5.9s
342:	learn: 0.3096336	total: 3.08s	remaining: 5.89s
343:	learn: 0.3095204	total: 3.09s	remaining: 5.89s
344:	learn: 0.3094205	total: 3.1s	remaining: 5.89s
345:	learn: 0.3091041	total: 3.11s	remaining: 5.88s
346:	learn: 0.3089416	total: 3.13s	remaining: 5.88s
347:	learn: 0.3087418	total: 3.13s	remaining: 5.87s
348:	learn: 0.3086786	total: 3.14s	remaining: 5.87s
349:	learn: 0.3084682	total: 3.15s	remaining: 5.85s
350:	learn: 0.30819

491:	learn: 0.2828109	total: 4.67s	remaining: 4.82s
492:	learn: 0.2826661	total: 4.68s	remaining: 4.82s
493:	learn: 0.2824037	total: 4.7s	remaining: 4.81s
494:	learn: 0.2822619	total: 4.71s	remaining: 4.8s
495:	learn: 0.2821302	total: 4.71s	remaining: 4.79s
496:	learn: 0.2819259	total: 4.72s	remaining: 4.78s
497:	learn: 0.2817979	total: 4.73s	remaining: 4.77s
498:	learn: 0.2817264	total: 4.74s	remaining: 4.76s
499:	learn: 0.2815691	total: 4.75s	remaining: 4.75s
500:	learn: 0.2814592	total: 4.76s	remaining: 4.74s
501:	learn: 0.2813281	total: 4.77s	remaining: 4.73s
502:	learn: 0.2811283	total: 4.77s	remaining: 4.72s
503:	learn: 0.2809904	total: 4.78s	remaining: 4.71s
504:	learn: 0.2808566	total: 4.79s	remaining: 4.7s
505:	learn: 0.2807426	total: 4.8s	remaining: 4.69s
506:	learn: 0.2805029	total: 4.81s	remaining: 4.67s
507:	learn: 0.2802407	total: 4.82s	remaining: 4.67s
508:	learn: 0.2800073	total: 4.83s	remaining: 4.66s
509:	learn: 0.2798563	total: 4.84s	remaining: 4.65s
510:	learn: 0.27

655:	learn: 0.2594908	total: 6.21s	remaining: 3.26s
656:	learn: 0.2593597	total: 6.23s	remaining: 3.25s
657:	learn: 0.2592417	total: 6.24s	remaining: 3.25s
658:	learn: 0.2592011	total: 6.26s	remaining: 3.24s
659:	learn: 0.2589289	total: 6.27s	remaining: 3.23s
660:	learn: 0.2587763	total: 6.29s	remaining: 3.22s
661:	learn: 0.2586997	total: 6.3s	remaining: 3.22s
662:	learn: 0.2585505	total: 6.31s	remaining: 3.21s
663:	learn: 0.2585473	total: 6.33s	remaining: 3.2s
664:	learn: 0.2584346	total: 6.34s	remaining: 3.19s
665:	learn: 0.2582562	total: 6.35s	remaining: 3.19s
666:	learn: 0.2580895	total: 6.36s	remaining: 3.18s
667:	learn: 0.2580798	total: 6.38s	remaining: 3.17s
668:	learn: 0.2579304	total: 6.39s	remaining: 3.16s
669:	learn: 0.2577577	total: 6.41s	remaining: 3.15s
670:	learn: 0.2576133	total: 6.42s	remaining: 3.15s
671:	learn: 0.2574371	total: 6.43s	remaining: 3.14s
672:	learn: 0.2572565	total: 6.44s	remaining: 3.13s
673:	learn: 0.2570733	total: 6.46s	remaining: 3.12s
674:	learn: 0.

825:	learn: 0.2392097	total: 8.25s	remaining: 1.74s
826:	learn: 0.2390840	total: 8.26s	remaining: 1.73s
827:	learn: 0.2389819	total: 8.26s	remaining: 1.72s
828:	learn: 0.2388654	total: 8.27s	remaining: 1.71s
829:	learn: 0.2387563	total: 8.28s	remaining: 1.7s
830:	learn: 0.2385784	total: 8.29s	remaining: 1.69s
831:	learn: 0.2384420	total: 8.29s	remaining: 1.68s
832:	learn: 0.2384367	total: 8.3s	remaining: 1.66s
833:	learn: 0.2383867	total: 8.31s	remaining: 1.65s
834:	learn: 0.2383532	total: 8.32s	remaining: 1.64s
835:	learn: 0.2382508	total: 8.33s	remaining: 1.63s
836:	learn: 0.2381167	total: 8.34s	remaining: 1.62s
837:	learn: 0.2380519	total: 8.35s	remaining: 1.61s
838:	learn: 0.2378960	total: 8.35s	remaining: 1.6s
839:	learn: 0.2377874	total: 8.36s	remaining: 1.59s
840:	learn: 0.2376542	total: 8.37s	remaining: 1.58s
841:	learn: 0.2375292	total: 8.38s	remaining: 1.57s
842:	learn: 0.2373854	total: 8.39s	remaining: 1.56s
843:	learn: 0.2372693	total: 8.4s	remaining: 1.55s
844:	learn: 0.23

992:	learn: 0.2229787	total: 10.1s	remaining: 71.5ms
993:	learn: 0.2228889	total: 10.2s	remaining: 61.3ms
994:	learn: 0.2227922	total: 10.2s	remaining: 51.1ms
995:	learn: 0.2226035	total: 10.2s	remaining: 40.9ms
996:	learn: 0.2224180	total: 10.2s	remaining: 30.7ms
997:	learn: 0.2223410	total: 10.2s	remaining: 20.5ms
998:	learn: 0.2221995	total: 10.2s	remaining: 10.2ms
999:	learn: 0.2220354	total: 10.2s	remaining: 0us


In [35]:
results['RNS_40'] = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 42.71%
roc: 85.56%
recall: 89.23%
precision: 28.08%


In [36]:
result = pd.DataFrame(results)
result

,simpl_model,RNS_25,RNS_10,RNS_40
0,0.561644,0.480558,0.502536,0.427140
1,0.727043,0.852588,0.839017,0.855590
2,0.488543,0.884137,0.865942,0.892323
3,0.660471,0.329948,0.353982,0.280770


***Вывод:*** В данном случае датасет довольно не сбалансирован, поэтому лучше всего сравилась по f1 простая модель. Также можно отметить что RNS справилась лучше при большем дисбалансе классов.